In [10]:
import numpy as np
import os
import skimage as sk
import skimage.io as skio
import scipy.ndimage as scnd
import matplotlib.pyplot as plt
from skimage.transform import rescale, resize
import cv2
import glob





In [14]:
def getGridImages(directory):
    #returns a hashmap of args fo reach imagem
    #returns an list of images
    map = {k:[] for k in range(290)}
    img_list = []
    count = 0
    for filename in os.listdir(directory):
            file_split = filename.split("_")
            #out, y, x, v, u, extension string type
            print(file_split)
            y_shift = file_split[1]
            x_shift = file_split[2]
            img_path = directory + filename 
            img = skio.imread(img_path)/255
            img = sk.util.img_as_float(img)
            map[count].extend([y_shift, x_shift])
            img_list.append(img)
            count += 1
    return np.array(img_list), map

In [20]:
#directories, scale range
argmap = {"chess": ["./chessboard/", "./output_images/chessboard/", [-4, 3]],
           "bracelet": ["./bracelet/", "./output_images/bracelet/", [-4,3]],
           "flower": ["./flower/", "./output_images/flower/", [-4,3]],
           "beans": ["./beans/", "./output_images/beans/", [-6,3]],
           "bottle": ["./bottle/", "./output_images/bottle/", [-4,3]]}

arg = "bottle"

directory = argmap[arg][0]
out_directory = argmap[arg][1]
scale_start = argmap[arg][2][0]
scale_end = argmap[arg][2][1]


img_list, map = getGridImages(directory)       
    

['out', '0', '0', '.jpg']
['out', '0', '1', '.jpg']
['out', '0', '2', '.jpg']
['out', '0', '3', '.jpg']
['out', '1', '0', '.jpg']
['out', '1', '1', '.jpg']
['out', '1', '2', '.jpg']
['out', '1', '3', '.jpg']
['out', '2', '0', '.jpg']
['out', '2', '1', '.jpg']
['out', '2', '2', '.jpg']
['out', '2', '3', '.jpg']
['out', '3', '0', '.jpg']
['out', '3', '1', '.jpg']
['out', '3', '2', '.jpg']
['out', '3', '3', '.jpg']


In [16]:
#CITATION: https://stackoverflow.com/questions/2777907/python-numpy-roll-with-padding
def depth_refocus_grid(img_list, shiftmap, alpha , out_directory, id, center = 8, both = False):
    shifted_imgs = []
    for i in range(len(img_list)):
        y_offset = int((center - int(shiftmap [i][0]))*alpha)
        x_offset = int((center - int(shiftmap [i][1]))*alpha)
        img = img_list[i]
        if both:
            img = np.roll(img, [-y_offset, -x_offset] , axis = (0, 1)) 
        else:
            img = np.roll(img, [y_offset, -x_offset] , axis = (0, 1)) 
        shifted_imgs.append(img)
    print("done")
    processed_image = np.mean(shifted_imgs, axis = 0)
    processed_image = sk.util.img_as_ubyte(processed_image)
    skio.imsave(out_directory + "/refocus/{scale}.png".format(scale = id), processed_image)
        

id = 0
for alpha in np.arange(scale_start, scale_end, 0.2):
    depth_refocus_grid(img_list, map, alpha, out_directory, id)
    id += 1



done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done
done


In [24]:
def aperture_readjustment(img_list, shiftmap, radius, id, center = 2):
    #select images that are all within a the given radius
    passed_imgs = []
    for i in range(len(img_list)):
        y_within_r = np.abs(int(center - int(shiftmap [i][0]))) <= radius
        x_within_r= np.abs(int(center - int(shiftmap [i][1]))) <= radius
        img = img_list[i]
        if y_within_r and x_within_r:
            passed_imgs.append(img)
    processed_image = np.mean(passed_imgs, axis = 0)
    processed_image = sk.util.img_as_ubyte(processed_image)
    skio.imsave(out_directory + "/aperture/{scale}.png".format(scale = id), processed_image)

id = 0
for radius in range(0,9):
    radius = int(radius)
    aperture_readjustment(img_list, map, radius, id)
    id+= 1


    
        

In [28]:
img_array = []
for filename in range(35):
   img = cv2.imread("./output_images/bottle/refocus/" + str(filename) + ".png")
   height, width, layers = img.shape
   size = (width,height)
   img_array.append(img)

out = cv2.VideoWriter('bottle.avi',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)

for i in range(len(img_array)):
   out.write(img_array[i])
out.release()